In [187]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split

In [189]:
earthquakes = pd.read_csv('earthquakes.csv')


In [190]:
earthquakes.head()

,magnitude,cdi,depth,distanceKM
0,4.8,6,4.0440,33
1,5.1,6,6.1279,34
2,3.7,4,10.9000,6
3,3.9,3,6.2024,58
4,4.1,3,8.8140,60


In [191]:
earthquakes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   magnitude   1137 non-null   float64
 1   cdi         1137 non-null   int64  
 2   depth       1137 non-null   float64
 3   distanceKM  1137 non-null   int64  
dtypes: float64(2), int64(2)
memory usage: 35.7 KB


In [192]:
earthquakes.shape

(1137, 4)

In [193]:
earthquakes.isna().sum()

magnitude     0
cdi           0
depth         0
distanceKM    0
dtype: int64

In [194]:
X = earthquakes[['cdi','depth','distanceKM']]
y = earthquakes['magnitude']

In [195]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [197]:
X_scaled[3]

array([ 0.02918438, -0.39947355,  0.10197061])

In [204]:
from scipy import stats
combined_data = np.column_stack((X_scaled, y))

z_scores = np.abs(stats.zscore(combined_data))

mask = (z_scores < 3).all(axis=1)

X_no_outliers = X_scaled[mask]
y_no_outliers = y[mask]


print(f"Original shape of X: {X.shape}")
print(f"Shape of X after removing outliers: {X_no_outliers.shape}")
print(f"Original shape of y: {y.shape}")
print(f"Shape of y after removing outliers: {y_no_outliers.shape}")

Original shape of X: (1137, 3)
Shape of X after removing outliers: (1054, 3)
Original shape of y: (1137,)
Shape of y after removing outliers: (1054,)


In [205]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(X_no_outliers, y_no_outliers, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training Data Shape:", X_train.shape)
print("Validation Data Shape:", X_val.shape)
print("Testing Data Shape:", X_test.shape)

Training Data Shape: (843, 3)
Validation Data Shape: (105, 3)
Testing Data Shape: (106, 3)


In [206]:
DENSE1_SIZE = 64
DENSE2_SIZE = 32
NUM_OF_EPOCHS = 200
BATCH_SIZE = 6

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(DENSE1_SIZE, activation='relu'),
    tf.keras.layers.Dense(DENSE2_SIZE, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [207]:
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['mae'])

In [208]:
from tensorflow.keras.callbacks import EarlyStopping



history = model.fit(
    X_train, 
    y_train,
    validation_data=(X_val, y_val),
    batch_size=BATCH_SIZE,
    epochs=NUM_OF_EPOCHS,
    verbose=1,
  
)


Epoch 1/200
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 18.1304 - mae: 3.9885 - val_loss: 1.3290 - val_mae: 0.9432
Epoch 2/200
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.5139 - mae: 0.9932 - val_loss: 0.9235 - val_mae: 0.8020
Epoch 3/200
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 1.0616 - mae: 0.8235 - val_loss: 0.8513 - val_mae: 0.7535
Epoch 4/200
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.8745 - mae: 0.7671 - val_loss: 0.7304 - val_mae: 0.7186
Epoch 5/200
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.8716 - mae: 0.7600 - val_loss: 0.6608 - val_mae: 0.6914
Epoch 6/200
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.7186 - mae: 0.7075 - val_loss: 0.6402 - val_mae: 0.6818
Epoch 7/200
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.7391 - mae: 0.7193 - val_loss: 0.6208 - val_mae: 0.6733
Epoch 8/200
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6498 - mae: 0.6640 - val_loss: 0.5728 - val_mae: 0.6324
Epoch 9/200
141/141 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms

In [209]:
train_loss, train_mae = model.evaluate(X_train, y_train, verbose=1)
val_loss, val_mae = model.evaluate(X_val, y_val, verbose=1)
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)

print(f"Training Loss: {train_loss:.2f}, Training MAE: {train_mae:.2f}")
print(f"Validation Loss: {val_loss:.2f}, Validation MAE: {val_mae:.2f}")
print(f"Test Loss: {test_loss:.2f}, Test MAE: {test_mae:.2f}")


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4811 - mae: 0.4712
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4250 - mae: 0.4959 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.5323 - mae: 0.5571 
Training Loss: 0.41, Training MAE: 0.46
Validation Loss: 0.43, Validation MAE: 0.50
Test Loss: 0.55, Test MAE: 0.57


In [210]:
model.save('EarthquakeMagnitudeModel.h5')

## Tflite conversion

In [212]:
import tensorflow as tf
import numpy as np

In [213]:
def representative_dataset():
    for i in range(100):  
        data = X_test[i].astype(np.float32) 
        yield [data.reshape(1, -1)]

In [214]:
model = tf.keras.models.load_model('EarthquakeMagnitudeModel.h5')

In [216]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\amrut\AppData\Local\Temp\tmptgor6880\assets


INFO:tensorflow:Assets written to: C:\Users\amrut\AppData\Local\Temp\tmptgor6880\assets


Saved artifact at 'C:\Users\amrut\AppData\Local\Temp\tmptgor6880'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 3), dtype=tf.float32, name='input_layer_3')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2294425439888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2294425439312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2294425440272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2294425439696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2294425441040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2294425440464: TensorSpec(shape=(), dtype=tf.resource, name=None)


C:\Users\amrut\anaconda3\Lib\site-packages\tensorflow\lite\python\convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [217]:
with open('EarthquakeMagnitudeModel.tflite', 'wb') as f:
    f.write(tflite_model)

In [218]:
interpreter = tf.lite.Interpreter(model_path="EarthquakeMagnitudeModel.tflite")
interpreter.allocate_tensors()

In [219]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print('input_details:\n', input_details)
print('output_details:\n', output_details)

input_details:
 [{'name': 'serving_default_input_layer_3:0', 'index': 0, 'shape': array([1, 3]), 'shape_signature': array([-1,  3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
output_details:
 [{'name': 'StatefulPartitionedCall_1:0', 'index': 11, 'shape': array([1, 1]), 'shape_signature': array([-1,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [220]:
input_shape = input_details[0]['shape']
input0_data = np.random.random_sample(input_shape)
input0_data = np.array(input0_data, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input0_data)

In [221]:
interpreter.invoke()
output0_data = interpreter.get_tensor(output_details[0]['index'])
print(output0_data)

[[5.5977316]]


In [222]:
output0_data_model = model.predict(input0_data)
print(output0_data_model)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
[[5.6126776]]


In [223]:
import time
import sys

def hex_to_c_array(hex_data, var_name):
    c_str = ""
    c_str += '#ifndef ' + var_name.upper() + '_H\n'
    c_str += "#define " + var_name.upper() + '_H\n\n'
    localtime = time.asctime(time.localtime(time.time()))
    c_str += " This model data was generated on " + localtime + '\n\n'
    c_str += " Tools used:\n Python:" + str(sys.version) + "\n Numpy:" + str(np.version.version) + \
             "\n TensorFlow:" + str(sys.version) + "\n Keras: " + str(tf.keras.__version__) + "\n\n"
    c_str += ' Model details are:\n'
    c_str += ' NUM_OF_EPOCHS = ' + str(NUM_OF_EPOCHS) + '\n'
    c_str += ' BATCH_SIZE    = ' + str(BATCH_SIZE) + '\n*/\n'
    c_str += '\nconst int ' + 'DENSE1_SIZE' + ' = ' + str(DENSE1_SIZE) + ';\n'
    c_str += 'const int ' + 'DENSE2_SIZE' + ' = ' + str(DENSE2_SIZE) + ';\n'      
    c_str += '\nconst unsigned int ' + var_name + '_len = ' + str(len(hex_data)) + ';\n'
    c_str += 'alignas(8) const unsigned char ' + var_name + '[] = {'
    hex_array = []
    for i, val in enumerate(hex_data):
        hex_str = format(val, '#04x')
        if (i + 1) < len(hex_data):
            hex_str += ','
        if (i + 1) % 12 == 0:
            hex_str += '\n'
        hex_array.append(hex_str)
    c_str += '\n' + format(''.join(hex_array)) + '\n};\n\n'
    c_str += '#endif //' + var_name.upper() + '_H'
    return c_str


In [224]:
with open("earthquake_model_esp32.h", 'w') as file:
    file.write(hex_to_c_array(tflite_model, "earthquake_model_esp32"))